## A Simple Model

In [1]:
from __future__ import print_function
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 5103)
/usr/local/lib/python2.7/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
data_path = "/home/ubuntu/data/"

### Training

Training Dataset (train and val)

In [3]:
img_size=(320, 180)

In [4]:
from keras import backend as K
K.set_image_dim_ordering('th')
def_batch_size = 96

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, 
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        data_path+"train/"+'1trainDir',
        target_size=img_size, 
        batch_size=def_batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        data_path+"train/"+'1valDir',
        target_size=img_size,
        batch_size=def_batch_size,
        class_mode='categorical')

Found 2984 images belonging to 8 classes.
Found 360 images belonging to 8 classes.


In [6]:
train_generator.class_indices

{'ALB': 0,
 'BET': 1,
 'DOL': 2,
 'LAG': 3,
 'NoF': 4,
 'OTHER': 5,
 'SHARK': 6,
 'YFT': 7}

Model Definition

In [7]:
nb_classes = 8

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

In [9]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, img_size[0], img_size[1]), border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(32, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

Model Training

In [11]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0)

In [12]:
model.fit_generator(
        train_generator,
        samples_per_epoch=2984,
        nb_epoch=30,
        validation_data=validation_generator,
        nb_val_samples=360,
        callbacks=[early_stopping], 
        verbose=1)

Epoch 1/30
2984/2984 [==============================] - 71s - loss: 1.8669 - acc: 0.3418 - val_loss: 1.8778 - val_acc: 0.1611
Epoch 2/30
2984/2984 [==============================] - 58s - loss: 1.7409 - acc: 0.3817 - val_loss: 1.8083 - val_acc: 0.4917
Epoch 3/30
2984/2984 [==============================] - 57s - loss: 1.6780 - acc: 0.4169 - val_loss: 1.8009 - val_acc: 0.5028
Epoch 4/30
2984/2984 [==============================] - 57s - loss: 1.6512 - acc: 0.4326 - val_loss: 1.7317 - val_acc: 0.5083
Epoch 5/30
2984/2984 [==============================] - 57s - loss: 1.6202 - acc: 0.4457 - val_loss: 1.7083 - val_acc: 0.4861
Epoch 6/30
2984/2984 [==============================] - 57s - loss: 1.5932 - acc: 0.4531 - val_loss: 1.5821 - val_acc: 0.4972
Epoch 7/30
2984/2984 [==============================] - 57s - loss: 1.5869 - acc: 0.4584 - val_loss: 1.7307 - val_acc: 0.4444
Epoch 8/30
2984/2984 [==============================] - 57s - loss: 1.5589 - acc: 0.4675 - val_loss: 1.5644 - val_acc:

Model Testing

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        data_path+'train/'+'1testDir',
        target_size=img_size,
        batch_size=def_batch_size,
        class_mode='categorical')

Found 433 images belonging to 8 classes.


In [15]:
print("Model Loss, Accuracy: ")
print(model.evaluate_generator(test_generator, val_samples=433))

Model Loss and Accuracy: 
[1.2404141965694295, 0.58429560044621209]


In [16]:
classes = {v: k for k, v in train_generator.class_indices.iteritems()}
print(classes)

{0: 'ALB', 1: 'BET', 2: 'DOL', 3: 'LAG', 4: 'NoF', 5: 'OTHER', 6: 'SHARK', 7: 'YFT'}


Confusion Matrix

In [17]:
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
Y_test = []
Y_preds = []
batch_ind = test_generator.batch_index
for X_batch, Y_batch in test_generator:
    y_pred = model.predict_classes(X_batch, verbose=0)
    Y_test.extend(np.argmax(Y_batch,axis=1))
    Y_preds.extend(y_pred)
    if test_generator.batch_index == batch_ind:
        break

In [19]:
names = range(len(classes.keys()))
for k,v in classes.iteritems():
    names[k] = v

In [20]:
print(classification_report(Y_test, Y_preds, target_names=names))

             precision    recall  f1-score   support

        ALB       0.58      0.92      0.71       191
        BET       0.00      0.00      0.00        19
        DOL       1.00      0.11      0.20         9
        LAG       0.00      0.00      0.00         5
        NoF       0.83      0.52      0.64        65
      OTHER       1.00      0.03      0.05        36
      SHARK       0.50      0.13      0.21        15
        YFT       0.47      0.42      0.44        93

avg / total       0.60      0.58      0.52       433



/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
print(confusion_matrix(Y_test, Y_preds))

[[175   0   0   0   0   0   0  16]
 [ 14   0   0   0   0   0   1   4]
 [  0   0   1   0   1   0   0   7]
 [  5   0   0   0   0   0   0   0]
 [ 31   0   0   0  34   0   0   0]
 [ 25   0   0   0   1   1   0   9]
 [  5   0   0   0   0   0   2   8]
 [ 48   0   0   0   5   0   1  39]]


Saving Trained Model

In [44]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open(data_path+"Simple_Keras-model0.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
    
# serialize weights to HDF5
model.save_weights(data_path+'Simple_Keras-model0.h5') 

# serialize class indices to a numpy file npy
np.save('/home/ubuntu/data/Simple_Keras-model0-classes.npy', classes)

### Predicting

Loading Trained Model

In [ ]:
from keras.models import model_from_yaml

In [ ]:
# load YAML and create model
yaml_file = open(data_path+'Simple_Keras-model1.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)
# load weights into new model
model.load_weights(data_path+'Simple_Keras-model1.h5")

In [ ]:
classes = np.load('/home/ubuntu/data/Simple_Keras-model1-classes.npy').item()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

Generating predictions on the test set

In [37]:
kaggle_datagen = ImageDataGenerator(rescale=1./255)
kaggle_generator = test_datagen.flow_from_directory(
        data_path+'test/',
        target_size=img_size,
        batch_size=def_batch_size,
        class_mode=None, 
        shuffle=False)

Found 1000 images belonging to 1 classes.


In [38]:
Y_preds = []
batch_ind = kaggle_generator.batch_index
for X_batch in kaggle_generator:
    y_pred = model.predict_proba(X_batch, verbose=0)
    Y_preds.extend(y_pred)
    if kaggle_generator.batch_index == batch_ind:
        break

In [42]:
test_filenames = kaggle_generator.filenames
test_id = [f[10:] for f in test_filenames]

In [34]:
import pandas as pd
import datetime
def create_submission(path, predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])
    result1.loc[:, 'image'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    print("Writing " + sub_file)
    result1.to_csv(path+sub_file, index=False)

In [43]:
create_submission(data_path, Y_preds, test_id, "Simple_Keras_1")

Writing submission_Simple_Keras_1_2016-11-19-22-11.csv


Public Score: 1.38706 (multi-class logarithmic loss.)